Lambda School Data Science, Unit 2: Predictive Modeling

# Kaggle Challenge, Module 3


## Assignment
- [x] [Review requirements for your portfolio project](https://lambdaschool.github.io/ds/unit2), then submit your dataset.
- [x] Continue to participate in our Kaggle challenge. 
- [x] Use scikit-learn for hyperparameter optimization with RandomizedSearchCV.
- [x] Submit your predictions to our Kaggle competition. (Go to our Kaggle InClass competition webpage. Use the blue **Submit Predictions** button to upload your CSV file. Or you can use the Kaggle API to submit your predictions.)
- [x] Commit your notebook to your fork of the GitHub repo.

## Stretch Goals

### Reading
- Jake VanderPlas, [Python Data Science Handbook, Chapter 5.3](https://jakevdp.github.io/PythonDataScienceHandbook/05.03-hyperparameters-and-model-validation.html), Hyperparameters and Model Validation
- Jake VanderPlas, [Statistics for Hackers](https://speakerdeck.com/jakevdp/statistics-for-hackers?slide=107)
- Ron Zacharski, [A Programmer's Guide to Data Mining, Chapter 5](http://guidetodatamining.com/chapter5/), 10-fold cross validation
- Sebastian Raschka, [A Basic Pipeline and Grid Search Setup](https://github.com/rasbt/python-machine-learning-book/blob/master/code/bonus/svm_iris_pipeline_and_gridsearch.ipynb)
- Peter Worcester, [A Comparison of Grid Search and Randomized Search Using Scikit Learn](https://blog.usejournal.com/a-comparison-of-grid-search-and-randomized-search-using-scikit-learn-29823179bc85)

### Doing
- Add your own stretch goals!
- Try other [categorical encodings](https://contrib.scikit-learn.org/categorical-encoding/). See the previous assignment notebook for details.
- In additon to `RandomizedSearchCV`, scikit-learn has [`GridSearchCV`](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html). Another library called scikit-optimize has [`BayesSearchCV`](https://scikit-optimize.github.io/notebooks/sklearn-gridsearchcv-replacement.html). Experiment with these alternatives.
- _[Introduction to Machine Learning with Python](http://shop.oreilly.com/product/0636920030515.do)_ discusses options for "Grid-Searching Which Model To Use" in Chapter 6:

> You can even go further in combining GridSearchCV and Pipeline: it is also possible to search over the actual steps being performed in the pipeline (say whether to use StandardScaler or MinMaxScaler). This leads to an even bigger search space and should be considered carefully. Trying all possible solutions is usually not a viable machine learning strategy. However, here is an example comparing a RandomForestClassifier and an SVC ...

The example is shown in [the accompanying notebook](https://github.com/amueller/introduction_to_ml_with_python/blob/master/06-algorithm-chains-and-pipelines.ipynb), code cells 35-37. Could you apply this concept to your own pipelines?


### BONUS: Stacking!

Here's some code you can use to "stack" multiple submissions, which is another form of ensembling:

```python
import pandas as pd

# Filenames of your submissions you want to ensemble
files = ['submission-01.csv', 'submission-02.csv', 'submission-03.csv']

target = 'status_group'
submissions = (pd.read_csv(file)[[target]] for file in files)
ensemble = pd.concat(submissions, axis='columns')
majority_vote = ensemble.mode(axis='columns')[0]

sample_submission = pd.read_csv('sample_submission.csv')
submission = sample_submission.copy()
submission[target] = majority_vote
submission.to_csv('my-ultimate-ensemble-submission.csv', index=False)
```

In [5]:
DATA_PATH = 'https://raw.githubusercontent.com/LambdaSchool/DS-Unit-2-Kaggle-Challenge/master/data/'

In [60]:
import pandas as pd
from sklearn.model_selection import train_test_split

# Merge train_features.csv & train_labels.csv
train = pd.merge(pd.read_csv(DATA_PATH+'waterpumps/train_features.csv'), 
                 pd.read_csv(DATA_PATH+'waterpumps/train_labels.csv'))

# Read test_features.csv & sample_submission.csv
test = pd.read_csv(DATA_PATH+'waterpumps/test_features.csv')
sample_submission = pd.read_csv(DATA_PATH+'waterpumps/sample_submission.csv')

train.shape, test.shape

((59400, 41), (14358, 40))

In [61]:
import numpy as np

def wrangle(X):
  """Wrangle train, validate, and test in the same way."""

  # Prevent SettingWithCopyWarning
  X = X.copy()

  # About 3% of the time, latitude has small values near zero,
  # outside Tanzania, so we'll treat these values as zero.
  X['latitude'] = X['latitude'].replace(-2e-08, 0)

  # Replace the null values in categorical features with 'unkown'.
#   X = X.fillna('unknown')

  # When columns have zeroes that shouldn't, they're like null values.
  # So we'll replace zeroes with nulls, and impute missing values later.
  # Also create a 'missing indicator' column, because the fact that values are
  # missing may be a predictive signal.
  cols_with_zeroes = ['longitude', 'latitude', 'construction_year',
                      'gps_height', 'population']
  for col in cols_with_zeroes:
    X[col] = X[col].replace(0, np.nan)
    X[col+'_MISSING'] = X[col].isnull()

  # Drop duplicate columns. Drop 'recorded_by' (never varies) and 'id' (always varies).
  dropped = ['quantity_group', 'payment_type', 'recorded_by', 'id']
  X = X.drop(columns=dropped)

  # Convert 'date_recorded' to datetime.
  X['date_recorded'] = pd.to_datetime(X['date_recorded'], infer_datetime_format=True)

  # Extract components from 'date_recorded', then drop the original column.
  X['year_recorded'] = X['date_recorded'].dt.year
  X['month_recorded'] = X['date_recorded'].dt.month
  X['day_recorded'] = X['date_recorded'].dt.day
  X = X.drop(columns='date_recorded')

  # Engineer feature: how many years from 'construction_year' to 'date_recorded'.
  X['years'] = X['year_recorded'] - X['construction_year']
  X['years_missing'] = X['years'].isnull()

  # Return the wrangled dataframe.
  return X

In [62]:
train = wrangle(train)
test = wrangle(test)

target = 'status_group'

# Arrange data into X features matrix and y target vector
# so we use *all* features, including high-cardinality categoricals
X_train = train.drop(columns=target)
y_train = train[target]
X_test = test

In [63]:
import category_encoders as ce
from sklearn.impute import SimpleImputer
from sklearn.pipeline import make_pipeline
from lightgbm import LGBMClassifier
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from scipy.stats import randint, uniform
from sklearn.ensemble import RandomForestClassifier

In [68]:
pipeline = make_pipeline(
    ce.OrdinalEncoder(), 
    SimpleImputer(), 
    RandomForestClassifier(random_state=42, n_jobs=-1)
)

param_distributions = { 
    'simpleimputer__strategy': ['mean', 'median'], 
    'randomforestclassifier__n_estimators': randint(50, 500), 
    'randomforestclassifier__max_depth': [5, 10, 15, 20, None],  
}

# If you're on Colab, decrease n_iter & cv parameters
search = RandomizedSearchCV(
    pipeline, 
    param_distributions=param_distributions, 
    n_iter=10, 
    cv=3, 
    scoring='accuracy', 
    verbose=10, 
    return_train_score=True, 
    n_jobs=-1
)

search.fit(X_train, y_train);

Fitting 3 folds for each of 10 candidates, totalling 30 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   2 tasks      | elapsed:   31.4s
[Parallel(n_jobs=-1)]: Done   9 tasks      | elapsed:  1.3min
[Parallel(n_jobs=-1)]: Done  19 out of  30 | elapsed:  2.6min remaining:  1.5min
[Parallel(n_jobs=-1)]: Done  23 out of  30 | elapsed:  2.7min remaining:   49.6s
[Parallel(n_jobs=-1)]: Done  27 out of  30 | elapsed:  3.0min remaining:   20.2s
[Parallel(n_jobs=-1)]: Done  30 out of  30 | elapsed:  3.1min finished


In [69]:
print('Best hyperparameters', search.best_params_)
print('Cross-validation Accuracy', -search.best_score_)

Best hyperparameters {'randomforestclassifier__max_depth': 20, 'randomforestclassifier__n_estimators': 499, 'simpleimputer__strategy': 'median'}
Cross-validation Accuracy -0.8088552188552188


In [70]:
pd.DataFrame(search.cv_results_).sort_values(by='rank_test_score')

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_randomforestclassifier__max_depth,param_randomforestclassifier__n_estimators,param_simpleimputer__strategy,params,split0_test_score,split1_test_score,split2_test_score,mean_test_score,std_test_score,rank_test_score,split0_train_score,split1_train_score,split2_train_score,mean_train_score,std_train_score
3,64.794923,0.488043,4.495643,0.423552,20,499,median,"{'randomforestclassifier__max_depth': 20, 'ran...",0.809899,0.810202,0.806465,0.808855,0.001695,1,0.957273,0.964066,0.965379,0.962239,0.003552
4,56.249932,1.889837,6.538429,1.153711,20,351,median,"{'randomforestclassifier__max_depth': 20, 'ran...",0.810455,0.809646,0.805657,0.808586,0.002097,2,0.957955,0.963990,0.964848,0.962264,0.003068
7,38.547563,1.654649,5.713385,1.478659,20,225,mean,"{'randomforestclassifier__max_depth': 20, 'ran...",0.810101,0.809242,0.806313,0.808552,0.001622,3,0.959141,0.963965,0.965657,0.962921,0.002760
2,24.778791,2.329414,5.307198,3.926657,20,191,median,"{'randomforestclassifier__max_depth': 20, 'ran...",0.808939,0.809040,0.806010,0.807997,0.001405,4,0.957828,0.963283,0.964520,0.961877,0.002907
1,21.660243,0.809957,3.779558,0.197619,None,138,mean,"{'randomforestclassifier__max_depth': None, 'r...",0.805657,0.806818,0.802121,0.804865,0.001997,5,0.999975,0.999975,1.000000,0.999983,0.000012
0,19.516675,2.609442,5.327386,1.732528,15,211,mean,"{'randomforestclassifier__max_depth': 15, 'ran...",0.797323,0.796919,0.797222,0.797155,0.000172,6,0.868939,0.880480,0.882222,0.877214,0.005894
8,33.832507,7.412486,2.798516,1.844193,10,454,median,"{'randomforestclassifier__max_depth': 10, 'ran...",0.757172,0.757626,0.761061,0.758620,0.001736,7,0.775530,0.777399,0.781288,0.778072,0.002398
6,45.008282,1.712939,5.476686,1.393426,10,492,median,"{'randomforestclassifier__max_depth': 10, 'ran...",0.756919,0.757727,0.760909,0.758519,0.001722,8,0.775707,0.777652,0.781136,0.778165,0.002246
5,34.021105,8.495324,7.599079,3.132955,10,383,mean,"{'randomforestclassifier__max_depth': 10, 'ran...",0.756768,0.757172,0.760960,0.758300,0.001888,9,0.776667,0.778182,0.782803,0.779217,0.002610
9,13.539452,0.532518,3.689466,0.164490,5,134,median,"{'randomforestclassifier__max_depth': 5, 'rand...",0.717323,0.718687,0.720202,0.718737,0.001176,10,0.724924,0.720758,0.720429,0.722037,0.002046


In [71]:
pipeline = search.best_estimator_

In [72]:
y_pred = pipeline.predict(X_test)

In [74]:
submission = sample_submission.copy()
submission['status_group'] = y_pred
submission.to_csv('sub6.csv', index=False)

In [77]:
# Filenames of your submissions you want to ensemble
files = ['sub3.csv', 'sub6.csv']

target = 'status_group'
submissions = (pd.read_csv(file)[[target]] for file in files)
ensemble = pd.concat(submissions, axis='columns')
majority_vote = ensemble.mode(axis='columns')[0]

submission = sample_submission.copy()
submission[target] = majority_vote
submission.to_csv('sub3_6_majority.csv', index=False)